### Downloading images from GEE

This code is used to retrieve and create the RGB, NDVI, DEM and Slope maps of the area around Palu, that was hit by a earthquake of Mw 7.5 on September 28th 2018 and resulted in at least 7063 landslides in the area.

The code is copied and adapted from Muhammad Aufaristama (https://code.earthengine.google.com/36f98798cad9ec8d3f7b7bb2e4112a02) for the area in Palu instead of Hokkaido, Japan. It is written in Javascript, which is the language the GEE uses.

In [ ]:
// Creating the satellite images for download for the Palu area in Indonesia

//Define datasets that used for landslides mapping
var S2 = ee.ImageCollection("COPERNICUS/S2")
var DEM = ee.Image("USGS/SRTMGL1_003")
var JRC = ee.Image("JRC/GSW1_3/GlobalSurfaceWater")

//Manual mapping inventory produced by Zhang et al 2018 using very high resolution image
//var table = ee.FeatureCollection("users/aufaristama/landslide_inventory_iburi");

//Define your area, in this case we use Palu, Sulawesi, Indonesia
var ROI =  ee.Geometry.Polygon(
        [[[120.1765296, -1.4991024],
          [120.1765296, -0.6142376],
          [119.6870732, -0.6142376],
          [119.6870732, -1.4991024]]], null, false);

//Zoom into your study area
Map.centerObject(ROI,10)

//Calculate slope from DEM
var terrain = ee.Terrain.products(DEM.select('elevation'))
var slope = terrain.select('slope').clip(ROI)        

//Create function to mask water from the image
function maskwater(image) {
  var water_mask = JRC.select(['max_extent']).lt(1)

  return image.updateMask(water_mask)
}

//Create function to mask cloud from the image
function maskS2clouds(image) {
  var qa = image.select('QA60');

  // Bits 10 and 11 are clouds and cirrus, respectively.
  var cloudBitMask = 1 << 10;
  var cirrusBitMask = 1 << 11;

  // Both flags should be set to zero, indicating clear conditions.
  var mask = qa.bitwiseAnd(cloudBitMask).eq(0)
      .and(qa.bitwiseAnd(cirrusBitMask).eq(0));

  return image.updateMask(mask).divide(10000);
}

//Create function to calculate spectral indices, in this case we will use only NDVI
function addindices(image) {
  var ndvi = image.normalizedDifference(['B8', 'B4']).rename(['ndvi']);
  return image.addBands(ndvi)
}

//Filter the pre-event image (Before the event)
var pre_event = S2.filterDate('2015-09-01', '2018-09-08')
.filterBounds(ROI)
.filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 30)
.map(maskwater)
.map(maskS2clouds)
.map(addindices)
print(pre_event, 'total_image_pre')

//Filter the post-event image (After the event)
var post_event = S2.filterDate('2018-09-29', '2020-09-01')
.filterBounds(ROI)
.filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 30)
.map(maskwater)
.map(maskS2clouds)
.map(addindices)
print(post_event, 'total_image_post')

//Calculate relative difference for change detection
//more details on https://nhess.copernicus.org/articles/21/1495/2021/nhess-21-1495-2021.html
var rd_subtract = post_event.max().subtract(pre_event.max());
var rd_add = pre_event.max().add(post_event.max());
var rd_sqrt = rd_add.sqrt();
var rd = (rd_subtract.divide(rd_sqrt)).multiply(100)

//Setup the rule/knowledge based thresholds to create binary raster of landslides
var landslide_binary = rd.select('ndvi').gt(-64).and(rd.select('ndvi').lt(-25)).and(slope.gt(10)).selfMask()

//Convert the binary into vector(shape_files)
var landslide_inventories = landslide_binary.reduceToVectors({
  geometry: ROI,
  scale: 10,
  geometryType: 'polygon',
  eightConnected: false,
  labelProperty: 'zone',
  maxPixels: 1e8
});

//Exporting the landslide results to your own Google drive
Export.table.toDrive({
  collection: landslide_inventories,
  description:'inventories_iburi_2018',
  fileFormat: 'SHP'
});

//Display the results in Google Earth Engine
Map.addLayer(slope, { }, 'slope',1)
Map.addLayer(pre_event.median().clip(ROI), {bands: ['B4', 'B3', 'B2'], max: 0.17}, 'visible_pre event',0);
Map.addLayer(post_event.median().clip(ROI), {bands: ['B4', 'B3', 'B2'], max: 0.17}, 'visible_post event',0);
Map.addLayer(pre_event.max().select('ndvi').clip(ROI), {min: -1, max: 0.85}, 'NDVI_pre event',0);
Map.addLayer(post_event.max().select('ndvi').clip(ROI), {min: -1,max: 0.85}, 'NDVI_post event',0);
Map.addLayer(rd.select('ndvi').clip(ROI),{min: -72, max: 29},"Relative difference (RD) NDVI",1);
Map.addLayer(landslide_binary .clip(ROI),{palette: 'red'},"Landslide_binary",1);
//Map.addLayer(table,  { }, 'inventories_Zhang_et_al',0);
//Map.addLayer(landslide_inventories,  { }, 'inventories_shapefiles',1);

Export.image.toDrive({
  image:DEM.select('elevation'),
  region:ROI,
  description:'DEM_Palu',
  fileNamePrefix:'DEM_Palu',
  scale : 10,
  maxPixels: 1e10,
  fileFormat: 'GeoTIFF'});
  
Export.image.toDrive({
  image:slope,
  region:ROI,
  description:'Slope_Palu',
  fileNamePrefix:'Slope_Palu',
  scale : 10,
  maxPixels: 1e10,
  fileFormat: 'GeoTIFF'});

Export.image.toDrive({
  image:post_event.max().select('ndvi'),
  region:ROI,
  description:'Palu_post_event-ndvi',
  fileNamePrefix:'Palu_post_event-ndvi',
  scale : 10,
  maxPixels : 1e10,
  fileFormat:'GeoTIFF'});

Export.image.toDrive({
  image:pre_event.max().select('ndvi'),
  region:ROI,
  description:'Palu_pre_event-ndvi',
  fileNamePrefix:'Palu_pre_event-ndvi',
  scale : 10,
  maxPixels : 1e10,
  fileFormat:'GeoTIFF'});

Export.image.toDrive({
  image:pre_event.median().select('B.+').visualize(['B4', 'B3', 'B2']),
  region:ROI,
  description:'Palu_pre_event-image',
  fileNamePrefix:'Palu_pre_event-image',
  scale : 10,
  maxPixels : 1e10,
  fileFormat:'GeoTIFF'});

Export.image.toDrive({
  image:post_event.median().select('B.+').visualize(['B4', 'B3', 'B2']),
  region:ROI,
  description:'Palu_post_event-image',
  fileNamePrefix:'Palu_post_event-image',
  scale : 10,
  maxPixels : 1e10,
  fileFormat:'GeoTIFF'});
  

In [ ]:
#Smaller Polygon for smaller area:
#(so that Colab won't crash!)
#ROI from Hakan (see email)
var ROI =  ee.Geometry.Polygon(
        [[[120.0564896, -1.5221459],
          [120.3093757, -1.1656366],
          [120.1664096, -1.0645648],
          [119.8387642, -1.5221459]]], null, false);

#Rechthoek
var ROI =  ee.Geometry.Polygon(
        [[[120.3093757, -1.0645648],
          [120.3093757, -1.5221459],
          [119.8387642, -1.5221459],
          [119.8387642, -1.0645648]]], null, false);
var ROI =  ee.Geometry.Polygon(
        [[[120.3094442, -1.0645510],
          [120.3094442, -1.5221518],
          [119.8387290, -1.5221518],
          [119.8387290, -1.0645510]]], null, false);

var ROI =  ee.Geometry.Polygon(
        [[[120.3094442, -1.0645510],
          [120.3094442, -1.5221518],
          [119.8389389, -1.5221518],
          [119.8389389, -1.0645510]]], null, false);


#THIS ROI WORKS TO GET THE RIGHT DEM AND SLOPE WITH THE RGB AND NDVI OF THE FIRST RECHTHOEK ROI
var ROI =  ee.Geometry.Polygon(
        [[[120.3094442, -1.0645510],
          [120.3094442, -1.5221518],
          [119.8388193, -1.5221518],
          [119.8388193, -1.0645510]]], null, false);


In [ ]:
#HOKKAIDO JAPAN
#FOR DEM & SLOPE AT 30M RESOLUTION WITH THE RGB+NDVI AT 30M FOR NORMAL ROI! :D
var ROI =  ee.Geometry.Polygon(
        [[[141.78777264285623, 42.9216532],
          [141.78777264285623, 42.5723893718531],
          [142.1825938098484, 42.5723893718531],
          [142.1825938098484, 42.9216532]]], null, false);